# Example Utilization
This section demonstrates the basic utilization of ForceFinder on a basic beam example. The examples cover:

- Basic use and workflow with the SPR object
- Using some of the miscellaneous techniques:
    - DOF weightings and normalization
    - The reduce drives update
    - Drive attenuation
    - Response limiting

In [1]:
import sdynpy as sdpy
import numpy as np

(sec:example_beam_system)=
## Creating the Example Beam System
All the examples use the simple beam system that is shown in the image below. Each dot in the image represents a node in the beam model. It is connected to the ground via translating and torsion springs (note that the torsion springs aren't shown in the image). The red arrows (below the beam) are the excitation locations and directions. The blue arrows (above the beam) are the response locations and directions that are used in the force estimation. 

```{note}
The nodes are labeled 101-109 from left to right and the translating direction (vertical on the page) is Z+.
```

```{figure} images/beam_system.svg
:alt: Example L-curve
:align: center
```

The beam is modeled with standard functions in SDynPy using Euler-Bernoulli beam elements, where the axial degrees of freedom (DOFs) have been excluded. The beam properties are:

- Beam length - 4 m
- Beam width - 0.02 m
- Beam height - 0.015 m
- Material properties - aluminum
- Translating spring stiffness to ground - 1e9 N/m
- Torsion spring stiffness to ground - 1e2 Nm/radian
- Beam damping ratio - 4% modal damping, note that the modal damping is added to the beam prior to adding the springs to ground, meaning that the damping ratio for the system will be different than this damping ratio

The resulting stiffness matrices saved in a SDynPy `System` object, which includes the mass, damping, and stiffness matrices.

In [2]:
youngs_modulus = 69.8e9 # pascals
density = 2700 # kg/m^3
poissons_ratio = 0.33
translating_spring = 1e9 #N/m
torsion_spring = 1e2 #Nm/rad
beam_damping_ratio = 0.04 # ratio
    
# Basic beam properties
beam_length = 4 # meters
beam_width = 0.02 # meters
beam_height = 0.015 # meters
number_nodes = 9

beam_coordinate = sdpy.coordinate_array(node=np.arange(number_nodes)[...,np.newaxis]+101, direction=[3,5]).flatten()

beam_stiffness, beam_mass = sdpy.beam.beamkm_2d(beam_length, beam_width, beam_height, 
                                                number_nodes, youngs_modulus, density, 
                                                poissons_ratio, axial=False)

beam_system = sdpy.System(beam_coordinate, beam_mass, beam_stiffness)
beam_system.assign_modal_damping(beam_damping_ratio)
    
beam_system.stiffness[0,0] += translating_spring # N/m, translating spring to ground on left side
beam_system.stiffness[1,1] += torsion_spring # Nm/rad, torsion spring to ground on left side 

beam_system.stiffness[-2,-2] += translating_spring # N/m, translating spring to ground on right side
beam_system.stiffness[-1,-1] += torsion_spring # Nm/rad, torsion spring to ground on right side 

### Beam Modal Properties
The beam system modal properties are shown in the table below.

In [3]:
print(beam_system.eigensolution())

   Index,  Frequency,    Damping,     # DoFs
    (0,),     2.5358,    1.5364%,         18
    (1,),     9.0563,    2.4143%,         18
    (2,),    19.9007,    3.1257%,         18
    (3,),    35.1521,    3.3233%,         18
    (4,),    54.9795,    3.4809%,         18
    (5,),    79.6855,    3.5745%,         18
    (6,),   109.6482,    3.6710%,         18
    (7,),   154.1341,    3.6553%,         18
    (8,),   191.9346,    3.6277%,         18
    (9,),   244.8775,    3.6781%,         18
   (10,),   309.2425,    3.7002%,         18
   (11,),   387.0350,    3.7040%,         18
   (12,),   478.9956,    3.6912%,         18
   (13,),   579.7017,    3.6604%,         18
   (14,),   668.1026,    3.6172%,         18
   (15,),   704.5058,    3.7964%,         18
   (16,), 29435.8118,    0.0646%,         18
   (17,), 29436.8999,    0.0647%,         18



## Beam System Geometry
The beam geometry is saved into a SDynPy `Geometry`. 

In [4]:
beam_locations = np.array([[0,   0, 0], 
                           [0.5, 0, 0],
                           [1, 0, 0], 
                           [1.5, 0, 0],
                           [2,   0, 0], 
                           [2.5, 0, 0],
                           [3, 0, 0], 
                           [3.5, 0, 0],
                           [4,   0, 0]], dtype=float)

system_cs = sdpy.geometry.coordinate_system_array()

beam_nodes = sdpy.geometry.node_array(id = np.unique(beam_coordinate.node), coordinate = beam_locations)
beam_fem = sdpy.geometry.Geometry(node = beam_nodes, coordinate_system = system_cs)

beam_fem.add_traceline(np.unique(beam_coordinate.node))

## Saving the SDynPy Objects
The SDynPy System and Geometry objects are saved into the documentation folder of the package, so they can be used in several examples.

In [5]:
beam_system.save('./example_system/example_system.npz')
beam_fem.save('./example_system/example_geometry.npz')